In [1]:
# installation of Weights & Biases
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.5 MB/s eta 0:00:00


In [2]:
# installation of pytest
!pip install pytest pytest-sugar

In [3]:
# Importing libraries
import wandb
import numpy as np
import pandas as pd

In [4]:
# wandb login
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
# create test file
%%file test_data.py

import pytest
import wandb
import pandas as pd
import numpy as np
import os
from PIL import Image

# initializing a wandb run to check preprocessing
run = wandb.init(entity="flamigos", project="cnn_animation", job_type="data_check")

@pytest.fixture(scope="session")
def data():
    artifact = run.use_artifact('flamigos/cnn_animation/label_data:latest', type='clean_data').file()
    artifact2 = run.use_artifact('flamigos/cnn_animation/img_data:latest', type='clean_data').file()
    labels = pd.read_csv(artifact)
    data_img = np.load(artifact2)

    return labels, data_img


def test_data_length(data):
    # check if we have a required amount of data and if the two artifects have the same size
    labels, data_img = data
    assert (len(labels) > 8800) and (len(data_img) > 8800) and (len(labels) == len(data_img))


def test_image_data_shape(data):
    # check if all values of the data_img artifact have the same shape
    labels, data_img = data
    for i in data_img:
      assert i.shape == data_img[0].shape


def test_label_data_types(data):
    # check if all values of the labels artifact have the same type
    labels, data_img = data
    for i in range(len(labels["animation Type"])):
      assert type(labels["animation Type"][i]) == type(labels["animation Type"][0])


def test_img_data_types(data):
    # check if all values of the data_img artifact have the same type
    labels, data_img = data
    for i in range(len(data_img)):
      assert type(data_img[i]) == type(data_img[0])


def test_image_data_valid(data):
    # check if the data of img_dat is a valid image
    # with this test we dont need a null check in the img_data!
    labels, data_img = data
    for i in data_img:
        img = i*255
        img = img.astype(np.uint8)
        image = Image.fromarray(img)
        assert (image.size == (128, 128) and image.mode == 'RGBA')


def test_label_range(data):
    # test whether category values are within an expected range of values
    labels, data_img = data
    assert labels["animation Type"].isin([0, 1]).all()


def test_null_values_from_labels(data):
    # test whether there is missing data in the data frame
    labels, data_img = data
    assert labels.isnull().any().any() == False

    # finish the run
    run.finish()

Writing test_data.py


In [6]:
# run test file
!pytest . -vv

Test session starts (platform: linux, Python 3.10.12, pytest 7.4.3, pytest-sugar 0.9.7)
cachedir: .pytest_cache
rootdir: /content
plugins: sugar-0.9.7, anyio-3.7.1
collected 7 items                                                                                  

 test_data.py::test_data_length ✓                                                     14% █▌        
 test_data.py::test_image_data_shape ✓                                                29% ██▉       
 test_data.py::test_label_data_types ✓                                                43% ████▍     
 test_data.py::test_img_data_types ✓                                                  57% █████▊    
 test_data.py::test_image_data_valid ✓                                                71% ███████▎  
 test_data.py::test_label_range ✓                                                     86% ████████▋ 
 test_data.py::test_null_values_from_labels ✓                                        100% ██████████

Results (115.45s (0:01:55))